In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException  #(클릭시 없을때, 엘리멘트 자체가 없을떄, 엘리멘트가 상호작용을 못할때 )


#웹브라우저를 띄우지 않고 진행하기 위한 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

### news_scraping() : 뉴스 스크래핑 함수

In [2]:
def news_scraping(news_url, wd):
    
    press = wd.find_element(By.XPATH, '//*[@id="main_content"]/div[1]/div[1]/a/img').get_attribute('title')
    title = wd.find_element(By.ID, 'articleTitle').text
    datetime = wd.find_element(By.CLASS_NAME, 't11').text
    article = wd.find_element(By.ID, 'articleBodyContents').text
    article = article.replace("// flash 오류를 우회하기 위한 함수 추가", "")
    article = article.replace("function _flash_removeCallback() {}", "")
    article = article.replace("\n", "")
    article = article.replace("\t", "")
    
    good = wd.find_element(By.XPATH, '//*[@id="spiLayer"]/div[1]/ul/li[1]/a/span[2]').text
    warm = wd.find_element(By.XPATH, '//*[@id="spiLayer"]/div[1]/ul/li[2]/a/span[2]').text
    sad = wd.find_element(By.XPATH, '//*[@id="spiLayer"]/div[1]/ul/li[3]/a/span[2]').text
    angry = wd.find_element(By.XPATH, '//*[@id="spiLayer"]/div[1]/ul/li[4]/a/span[2]').text
    want = wd.find_element(By.XPATH, '//*[@id="spiLayer"]/div[1]/ul/li[5]/a/span[2]').text
    recommend = wd.find_element(By.XPATH, '//*[@id="toMainContainer"]/a/em[2]').text
    
#     print("뉴스: ", [title, press, datetime, article, good, warm, sad, angry, want, recommend, news_url])
    
    return [title, press, datetime, article, good, warm, sad, angry, want, recommend, news_url]
    

### comments_scraping(): 뉴스 댓글 스크래핑

In [3]:
def comments_scraping(news_url, wd):
    try:
        btn_view = wd.find_element(By.CLASS_NAME, 'u_cbox_btn_view_comment').click() #댓글 더 보기
        print("[댓글 더 보기]", end="")
        time.sleep(1)
        
        while True: #댓글 더보기 이후 더 보기를 계속 클릭해서 끝까지 가야하므로 while문을 써준다.
            wd.find_element(By.CLASS_NAME,'u_cbox_page_more').click() #더 보기
            print("[더 보기]", end="")
            time.sleep(1)
        
    except:
        pass
    
    btn_reply_list = wd.find_elements(By.CLASS_NAME, 'u_cbox_btn_reply') #답글 버튼 들고 옴
    for btn_reply in btn_reply_list:
        btn_reply.send_keys('\n') #.click()으로 변화가 없는 부분은 이렇게 처리 한다.
        print("[답글]", end="")
        time.sleep(1)
        
    print("[댓글 스크래핑]")
    
    comments_idx = 0
    comments_df = pd.DataFrame(columns=("Contents","Name","Datetime","Like","Dislike","URL"))
    
    
    comments = wd.find_elements(By.CLASS_NAME, 'u_cbox_comment_box') #댓글 박스 들고 옴
    for comment in comments:
        try:
            name = comment.find_element(By.CLASS_NAME, 'u_cbox_nick').text
            date = comment.find_element(By.CLASS_NAME, 'u_cbox_date').text
            contents = comment.find_element(By.CLASS_NAME, 'u_cbox_contents').text
            recomm = comment.find_element(By.CLASS_NAME, 'u_cbox_cnt_recomm').text
            unrecomm = comment.find_element(By.CLASS_NAME, 'u_cbox_cnt_unrecomm').text
            print(f"  댓글 #{comments_idx+1}:", [contents, name, date, recomm, unrecomm, news_url])
            comments_df.loc[comments_idx] = [contents, name, date, recomm, unrecomm, news_url]
            comments_idx += 1
        except NoSuchElementException:
#             print("[삭제되거나 부적절한 댓글]")
            continue
    
    return comments_df
        

### scrpaing() : 스크래핑 함수

In [4]:
def scraping(news_max = 2):
    wd = webdriver.Chrome('chromedriver', options=chrome_options)
    wd.implicitly_wait(3) #대기 시간
    
    # --- 검색을 위한 세팅 ---
    wd.execute_script('window.open("about:blank", "_blank");') #JS의 새 탭을 띄우는 용도
    tabs = wd.window_handles #탭 핸들링이 가능해 짐
    wd.switch_to.window(tabs[0]) #첫번째 탭으로 전환
    query = input("검색어 입력: ")
    search_url = "https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query=" + query
    wd.get(search_url)
    
    
    # --- Data-Frame ---
    news_idx = 0
    news_df = pd.DataFrame(columns=("Title","Press","DateTime","Article","Good","Warm","Sad","Angry","Want","Recommend","URL"))
    comments_df = pd.DataFrame()
    
    
    
    # -- 검색 이후 '네이버 뉴스' 만 스크래핑 ---
    while True:
        info_list = wd.find_elements(By.CLASS_NAME, 'info_group')
        for info in info_list:
            try:
                first_step = info.find_elements(By.TAG_NAME, 'a')
                news_url = first_step[1].get_attribute('href') #'네이버 뉴스' 의 링크를 가져 옴
                time.sleep(1)
                print(news_url)
            except:
                continue
            
            wd.switch_to.window(tabs[1])
            wd.get(news_url)

            news_df.loc[news_idx] = news_scraping(news_url, wd)
            news_idx += 1
        
            # --- 코멘트를 PD로 만들고 계속 합쳐서 만듬 ---
            df = comments_scraping(news_url, wd)
            comments_df = pd.concat([comments_df, df])
            
            if news_idx >= news_max:
                break
                
        if news_idx >= news_max:
            break
                
        
        # --- 페이징 ---
        try:
            wd.switch_to.window(tabs[0]) #탭[1]에서 스크래핑이 다 끝나면 다시 텝[0]으로 돌아오고 다음 페이지로 이동 시킴
            wd.find_element(By.CLASS_NAME, 'btn_next').click()
            time.sleep(1)
        except:
            break
    wd.close()
    
    return news_df, comments_df

In [5]:
news_df, comments_df = scraping()

검색어 입력: 인공지능
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=030&aid=0002993312
[댓글 스크래핑]
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=215&aid=0001008785
[댓글 스크래핑]


In [6]:
news_df

,Title,Press,DateTime,Article,Good,Warm,Sad,Angry,Want,Recommend,URL
0,"코웨이 노블 정수기 품질, 인공지능이 인증했다",전자신문,2022.01.12. 오후 2:02,코웨이는 프리미엄 디자인 노블 정수기가 국내 최초로 'AI+(에이아이플러스)' 인공...,0,0,0,0,0,,https://news.naver.com/main/read.naver?mode=LS...
1,"얼굴인식 인공지능(AI) 스타트업 메사쿠어컴퍼니, 80억 시리즈A 투자유치",한국경제TV,2022.01.12. 오전 10:00,제1금융권 모바일뱅킹에 얼굴인식 기술 적용모바일운전면허증 구축사업 얼굴인식 솔루션 ...,0,0,0,0,0,,https://news.naver.com/main/read.naver?mode=LS...


In [7]:
comments_df

,Contents,Name,Datetime,Like,Dislike,URL
